In [2]:
import re
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time as ts

In [3]:
List_urls_pd=pd.read_excel(r"E:\Jupyter Notebook and Codes\Jupyter Notebooks Folder\Reddit_Comments\List_Reddit_Links.xlsx")

In [4]:
List_urls=list(List_urls_pd['List_urls'])

In [5]:
List_urls

['https://www.reddit.com/r/cars/comments/8axmrm/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8fyopa/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/7trmi6/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8ogg6z/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8cmqxg/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8hmpcx/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/7oxvox/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8l00uc/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8jbff6/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/80cayx/weekly_what_car_should_i_buy_megathread/',
 'https://www.reddit.com/r/cars/comments/8258fa/weekly_what_car_should_i_buy_megathread/',

In [6]:
driver=webdriver.Chrome(r"E:\Jupyter Notebook and Codes\Jupyter Notebooks Folder\chromedriver.exe")

In [7]:
filename_list=[]
html_source={}
for url in List_urls:
    link=re.search(r'\/comments\/(.*?)\/',url,re.I).group(1)
    print (link,List_urls.index(url))
    filename_list.append(re.search(r'\/comments\/(.*?)\/',url,re.I).group(1))
    driver.get(url)
    ts.sleep(7)
    i=0
    while True:
        if i>100: break
        try:
            elements=driver.find_elements_by_xpath("//*[contains(@id,'moreComments-t1_e0')]")
            for element in elements:
                element.click()
                print ("it worked")
        except:
            try:
                elements=driver.find_elements_by_xpath("//*[contains(@id,'more_t1_e0')]")
                for element in elements:
                    element.click()
                    print ("it worked")
            except NoSuchElementException:
                break
        i=i+1
        #print (i)
        ts.sleep(0.5)
    html_source[link]=driver.page_source
    Authors_Comments_initial=re.findall(r'new \(suggested\)(.*?)<div class="s1axw53s-8 kROdPU">',html_source[link],re.I)
    Authors_Comments=re.findall(r'</button>(.*?)<div class="s1axw53s-8 kROdPU">',html_source[link],re.I)
    Authors_Comments_final=Authors_Comments_initial+Authors_Comments
    ip_series=pd.Series(Authors_Comments)
    ip=pd.DataFrame(ip_series,columns=['Original_html'])
    ip['Author_Name']=''
    ip['Content']=''
    ip['Type_of_Content']='Sub_Content'
    ip['Time']=''
    ip['Main_Document_id']=''
    ip['url']=url
    for each in Authors_Comments:
        index=Authors_Comments.index(each)
        #print (index)
        if re.search(r'^(?!.*removed|deleted)',each,re.I):
            try:
                ip.ix[index,'Author_Name']=re.search(r'<a class=".*?" href="/user/(.*?)">',each,re.I).group(1)
            except:
                print ("Author_Name_deleted")
                ip.ix[index,'Author_Name']="Author_Name_deleted"
                pass
            if re.search(r'<div class="s136il31-0 cMWqxb"',each,re.I):
                ip.ix[index,'Type_of_Content']='Main_Content'
                id_main=re.search(r'id="(t1.*?)"',each,re.I).group(1)
            ip.ix[index,'Main_Document_id']=id_main
            temp0=re.sub(re.compile(r'.*href="/user.*?point',re.I),'',each)
            temp=re.sub(re.compile('\<.+?\>',re.I),'. ',temp0) #isolating just the comments part
            temp2=re.sub(re.compile(r'\xe2\x80\x99',re.I),'',temp) #replacing apostrophe with ` since apostrophe is illegible
            temp3=re.sub(re.compile(r'\xe2\x80\x9c',re.I),'',temp2)#replacing opening hex version of " with proper "
            temp4=re.sub(re.compile(r'\xe2\x80\x9d',re.I),'',temp3)#replacing closing hex version of " with proper "
            temp5=re.sub(re.compile(r'&#x27;',re.I),'',temp4)#replacing &#x27; with `
            temp6=re.sub(re.compile(r'(\.\s){2,}'),'. ',temp5)
            temp7=re.sub(re.compile(r':.'),':',temp6)
            ip.ix[index,'Time']=re.search(r'([a-zA-Z0-9 ]+ago)',temp7,re.I).group(1)
            temp8=re.sub(re.compile(r'(.*[a-zA-Z0-9 ]+ago\.)',re.I),'',temp7)
            ip.ix[index,'Content']=temp8
        else:
            print ("Removed ",index)  
    ip.drop(['Original_html'],axis=1,inplace=True)
    if List_urls.index(url)==0:
        op=ip.copy()
    else:
        op=op.append(ip,ignore_index=True)
op.to_csv(r'E:\Jupyter Notebook and Codes\Jupyter Notebooks Folder\Reddit_Comments\Final_reddits.csv')
print (len(op))

8axmrm 0


C:\Users\welcome\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Author_Name_deleted
8fyopa 1
Removed  156
Removed  157
7trmi6 2
Removed  27
Removed  28
Removed  29
Removed  30
Author_Name_deleted
Author_Name_deleted
Removed  62
Removed  63
Author_Name_deleted
Author_Name_deleted
Removed  138
Removed  139
Author_Name_deleted
Author_Name_deleted
Removed  162
Removed  163
Removed  171
Removed  172
8ogg6z 3
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
it worked
8cmqxg 4
Removed  37
8hmpcx 5
Removed  6
Removed  7
Removed  140
Removed  141
7oxvox 6
Removed  30
Removed  31
Author_Name_deleted
Author_Name_deleted
8l00uc 7
Author_Name_deleted
Author_Name_deleted
Author_Name_deleted
Removed  168
Removed  169
8jbff6 8
80cayx 9
Removed  5
Removed  6
Removed  95
Removed  96
Author_Name_deleted
8258fa 10
Removed  132
Removed  133
7ylyo0 11
Removed  8
Removed  9
Removed  24
Removed  25
7j1kj